# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [36]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [37]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
file_path_list = []
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list += glob.glob(os.path.join(root,'*.csv'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [38]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Example of row above is as following:


In [39]:
full_data_rows_list[2]

['Camila',
 'Logged In',
 'Tucker',
 'M',
 '1',
 'Garrison',
 '230.81751',
 'free',
 'Oxnard-Thousand Oaks-Ventura, CA',
 'PUT',
 'NextSong',
 '1.54083E+12',
 '555',
 'Abrazame  (Version Acustica)',
 '200',
 '1.54251E+12',
 '40']

#### The list obtained from executing the cell above is as following:
<p>['Camila',
 'Logged In',
 'Tucker',
 'M',
 '1',
 'Garrison',
 '230.81751',
 'free',
 'Oxnard-Thousand Oaks-Ventura, CA',
 'PUT',
 'NextSong',
 '1.54083E+12',
 '555',
 'Abrazame  (Version Acustica)',
 '200',
 '1.54251E+12',
 '40']</p>

In [40]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6987


# Part II. Apache Cassandra coding. 

#### Part I creates the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [52]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [53]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra_practice 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [54]:
try:
    session.set_keyspace('cassandra_practice')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

- Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

- Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
- Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
This query retrieves data based on `sessionId` and `itemInSession`. This implies that both keys are good candidates for a partition key. And since both keys guarantee the uniqueness condition, primary key should be `PRIMARY KEY ((sessionId, itemInSession))`. 

In [44]:
create_query = 'CREATE TABLE IF NOT EXISTS item_library'
create_query = create_query + '(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY ((sessionId, itemInSession)))'
try:
    session.execute(create_query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO item_library (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]), line[0], line[9], float(line[5])))
   

In [45]:
query = "SELECT artist, song, length FROM item_library WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, '|', row.song, '|', row.length) 

Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875


#### The solution from executing the cell above is:
Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875

#### Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)for userid = 10, sessionid = 182
This query retrieves data based on `userId` and `sessionId`. This implies that both keys are good candidates for a partition key. However, it is not sufficient to use `userid` and `sessionid` for primary key because they are not unique. Thus, more columns are required for primary key. Fortunately, we want to sort song titles by `ItemInSession`, and the combination of `userId`, `sessionId` and `itemInSession` satisfies the uniqueness condition. This makes `PRIMARY KEY ((userId, sessionId), itemInSession)` to be a reasonable choice. In other words, `itemInSession` is a clustering column that fulfills the uniqueness condition.

In [46]:
create_query = 'CREATE TABLE IF NOT EXISTS user_session_library'
create_query = create_query + '(userId int, sessionId int, artist text, firstName text, lastName text, song text,  '
create_query = create_query +  'itemInSession int, PRIMARY KEY ((userId, sessionId), itemInSession))'
try:
    session.execute(create_query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_library (userId, sessionId, artist, firstName, lastName, song, itemInSession)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[1], line[4], line[9], int(line[3])))
            
   
                    

In [47]:
query = "SELECT artist, song, firstname, lastname FROM user_session_library WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, '|', row.song, '|',row.firstname,'|', row.lastname)

Down To The Bone | Keep On Keepin' On | Sylvie | Cruz
Three Drives | Greece 2000 | Sylvie | Cruz
Sebastien Tellier | Kilometer | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz


#### The solution from executing the cell above is:
<p> Down To The Bone | Keep On Keepin' On | Sylvie | Cruz </p>
<p>Three Drives | Greece 2000 | Sylvie | Cruz</p>
<p>Sebastien Tellier | Kilometer | Sylvie | Cruz</p>
<p>Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz</p>

#### Query 3:  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
This query retrieves data based on song titles. Thus, a sound choice for partition key would be `song`. In order to ensure uniqueness, I chose to augment `song` with `userId`. As this project is a hypothetical example, I can proceed by assuming that the uniqueness indeed holds. However, a real world project would require a well-thought-out selection. In conclusion, the reasonable choice would be `PRIMARY KEY ((song), userId)`.

In [48]:
create_query = 'CREATE TABLE IF NOT EXISTS song_library'
create_query = create_query + '(song text, firstName text, lastName text, userId int, PRIMARY KEY ((song), userId))'
try:
    session.execute(create_query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (song, userId, firstName, lastName )"
        query = query + "VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]), line[1], line[4]))
            


                    

In [49]:
query = "SELECT firstname, lastname FROM song_library WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.firstname, '|', row.lastname) 

Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


#### The solution from executing the cell above is:
<p>Jacqueline | Lynch</p>
<p>Tegan | Levine</p>
<p>Sara | Johnson</p>

### Drop the tables before closing out the sessions

In [50]:
query = "drop table item_library"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_session_library"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_library"
try:
    session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection

In [51]:
session.shutdown()
cluster.shutdown()